# 🧠 Proyecto TB1: Optimización de Ensayo de Orquesta

**Curso:** Tópicos en Ciencias de Computación (2025-2)  
**Autores:**
    - [Nombre Alumno 1]
    - [Nombre Alumno 2]
    - [Nombre Alumno 3]
    - [Nombre Alumno 4]
    
## 1. Introducción

Este notebook presenta la solución al problema de **Optimización del Horario de un Ensayo de Orquesta**. El objetivo es encontrar la secuencia óptima de 9 composiciones para minimizar el tiempo total de espera de 5 músicos.

El problema se ha modelado como un **Problema de Satisfacción de Restricciones con Optimización (CSOP)** y se ha resuelto utilizando Python.

## 2. Definición de los Datos del Problema

En esta sección, definimos todas las variables de entrada basadas en la descripción del problema:
-   La duración de cada composición.
-   La matriz de participación que indica qué músico toca en cada composición.

In [28]:
import itertools
import time

In [29]:
# --- Datos del Problema ---
# Índices 0-8 para Composiciones C1-C9
DURACIONES = [2, 4, 1, 3, 3, 2, 5, 7, 6]
MUSICOS = [
    [1, 0, 1, 1, 0, 1, 1, 0, 1],  # Músico 1
    [1, 0, 1, 1, 1, 1, 0, 1, 0],  # Músico 2
    [1, 1, 0, 0, 1, 0, 0, 1, 0],  # Músico 3
    [1, 0, 0, 0, 1, 0, 1, 0, 1],  # Músico 4
    [0, 1, 0, 1, 1, 1, 1, 0, 0],  # Músico 5
]

In [30]:
def calcular_espera_total(orden, duraciones, musicos):
    """Calcula el tiempo total de espera para un orden de ensayo dado."""
    timeline = {}
    tiempo_actual = 0
    for comp_idx in orden:
        inicio = tiempo_actual
        fin = inicio + duraciones[comp_idx]
        timeline[comp_idx] = {'inicio': inicio, 'fin': fin}
        tiempo_actual = fin

    espera_total = 0
    detalles_musicos = []
    
    for i, musico_toca in enumerate(musicos):
        piezas_del_musico = [c for c in orden if musico_toca[c] == 1]
        
        if not piezas_del_musico:
            continue

        llegada = timeline[piezas_del_musico[0]]['inicio']
        salida = timeline[piezas_del_musico[-1]]['fin']
        span = salida - llegada
        
        tiempo_activo = sum(duraciones[c] for c in piezas_del_musico)
        espera_musico = span - tiempo_activo
        espera_total += espera_musico
        
        detalles_musicos.append({
            'musico': i + 1, 'span': span, 'actividad': tiempo_activo, 'espera': espera_musico
        })
        
    return espera_total, detalles_musicos

## 3. Modelado del Problema



In [ ]:
def optimizar_ensayo_con_simetria_corregida():
    """
    Encuentra el orden óptimo mediante búsqueda exhaustiva, aplicando 
    restricciones y una ruptura de simetría robusta.
    """
    composiciones = range(len(DURACIONES))
    mejor_orden = None
    min_espera = float('inf')
    mejor_detalle = None

    for orden in itertools.permutations(composiciones):
        # Para facilitar la búsqueda de posiciones
        pos = {comp: i for i, comp in enumerate(orden)}

        # --- Filtro 1: Restricciones del problema ---
        if pos[1] > pos[7]:  # C2 (idx 1) debe ir antes de C8 (idx 7)
            continue
        if pos[5] != pos[4] + 1:  # C6 (idx 5) debe ir justo después de C5 (idx 4)
            continue

        # --- Filtro 2: Ruptura de Simetría---
        # Forzamos un orden relativo entre dos piezas no restringidas (C1 y C2)
        # para eliminar soluciones revertidas sin descartar la óptima.
        if pos[0] > pos[1]: # C1 (idx 0) debe ir antes de C2 (idx 1)
            continue
            
        espera_actual, detalle_actual = calcular_espera_total(orden, DURACIONES, MUSICOS)

        if espera_actual < min_espera:
            min_espera = espera_actual
            mejor_orden = orden
            mejor_detalle = detalle_actual
            
    return mejor_orden, min_espera, mejor_detalle

## 4. Ejecución y Análisis de Resultados

In [32]:
if __name__ == "__main__":
    start_time = time.time()
    
    orden_optimo, espera_minima, detalle_optimo = optimizar_ensayo_con_simetria_corregida()
    
    end_time = time.time()
    
    orden_legible = [f"C{c+1}" for c in orden_optimo]
    
    print("\nOrden óptimo de las composiciones:")
    print(f"   [{' -> '.join(orden_legible)}]")
    
    print("\nDesglose del tiempo por músico:")
    for detalle in detalle_optimo:
        print(f" - Músico {detalle['musico']}: Span = {detalle['span']:<2} min, "
              f"Actividad = {detalle['actividad']:<2} min => "
              f"Tiempo de espera = {detalle['espera']:<2} min")
    
    print()
    print(f"Tiempo Total de Espera Minimizado: {espera_minima} minutos")

    print(f"\nEstadísticas de la Búsqueda:")
    print(f"   - Tiempo de ejecución: {end_time - start_time:.4f} segundos")



Orden óptimo de las composiciones:
   [C9 -> C7 -> C1 -> C5 -> C6 -> C3 -> C4 -> C2 -> C8]

Desglose del tiempo por músico:
 - Músico 1: Span = 22 min, Actividad = 19 min => Tiempo de espera = 3  min
 - Músico 2: Span = 22 min, Actividad = 18 min => Tiempo de espera = 4  min
 - Músico 3: Span = 22 min, Actividad = 16 min => Tiempo de espera = 6  min
 - Músico 4: Span = 16 min, Actividad = 16 min => Tiempo de espera = 0  min
 - Músico 5: Span = 20 min, Actividad = 17 min => Tiempo de espera = 3  min

Tiempo Total de Espera Minimizado: 16 minutos

Estadísticas de la Búsqueda:
   - Tiempo de ejecución: 0.3944 segundos
